In [1]:
import os
import time
from pathlib import Path
import pandas as pd
import QuantLib as ql
import numpy as np
from datetime import datetime
from model_settings import ms
from itertools import product
from joblib import Parallel, delayed


pricing settings:
Actual/365 (Fixed) day counter
New York stock exchange calendar
compounding: continuous
frequency: annual



In [2]:
from model_settings import asian_option_pricer
help(asian_option_pricer)

Help on asian_option_pricer in module model_settings.asian_option_pricer object:

class asian_option_pricer(builtins.object)
 |  asian_option_pricer(seed=None)
 |
 |  Methods defined here:
 |
 |  __init__(self, seed=None)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |
 |  asian_option_price(self, s, k, r, g, w, averaging_type, n_fixings, fixing_frequency, past_fixings, kappa, theta, rho, eta, v0, calculation_datetime)
 |
 |  df_asian_option_price(self, df)
 |
 |  row_asian_option_price(self, row)
 |
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |
 |  __dict__
 |      dictionary for instance variables
 |
 |  __weakref__
 |      list of weak references to the object



In [3]:
def asian_option_price(s,k,r,g,w,averaging_type,n_fixings,fixing_frequency,past_fixings,kappa,theta,rho,eta,v0,calculation_datetime):
    s = float(s)
    k = float(k)
    r = float(r)
    g = float(g)
    rng = "pseudorandom" # could use "lowdiscrepancy"
    numPaths = 100000
    
    if w == 'call':
        option_type = ql.Option.Call 
    elif w == 'put':
        option_type = ql.Option.Put
    t = n_fixings*fixing_frequency
    
    calculation_date = ql.Date(calculation_datetime.day,calculation_datetime.month,calculation_datetime.year)
    
    periods = np.arange(fixing_frequency,t+1,fixing_frequency).astype(int)
    
    fixing_dates = [calculation_date + ql.Period(int(p),ql.Days) for p in periods]
    expiration_date = calculation_date + ql.Period(int(t),ql.Days)
    
    riskFreeTS = ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, float(r), ql.Actual365Fixed()))
    dividendTS = ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, float(g), ql.Actual365Fixed()))
    
    hestonProcess = ql.HestonProcess(riskFreeTS, dividendTS, ql.QuoteHandle(ql.SimpleQuote(s)), v0, kappa, theta, eta, rho)
    hestonModel = ql.HestonModel(hestonProcess)
    vanillaPayoff = ql.PlainVanillaPayoff(option_type, float(k))
    europeanExercise = ql.EuropeanExercise(expiration_date)
    
    if averaging_type == 'geometric':
        geometric_engine = ql.MCDiscreteGeometricAPHestonEngine(hestonProcess, rng, requiredSamples=numPaths,seed=123)
        geometricAverage = ql.Average().Geometric
        geometricRunningAccumulator = 1.0
        discreteGeometricAsianOption = ql.DiscreteAveragingAsianOption(
        	geometricAverage, geometricRunningAccumulator, past_fixings,
            fixing_dates, vanillaPayoff, europeanExercise)
        discreteGeometricAsianOption.setPricingEngine(geometric_engine)
        geometric_price = float(discreteGeometricAsianOption.NPV())
        return geometric_price
        
    elif averaging_type == 'arithmetic':
        arithmetic_engine = ql.MCDiscreteArithmeticAPHestonEngine(hestonProcess, rng, requiredSamples=numPaths)
        arithmeticAverage = ql.Average().Arithmetic
        arithmeticRunningAccumulator = 0.0
        discreteArithmeticAsianOption = ql.DiscreteAveragingAsianOption(
            arithmeticAverage, arithmeticRunningAccumulator, past_fixings, 
            fixing_dates, vanillaPayoff, europeanExercise)
        discreteArithmeticAsianOption.setPricingEngine(arithmetic_engine)
        arithmetic_price = float(discreteArithmeticAsianOption.NPV())
        return arithmetic_price
    else:
        print("invalid Asian option averaging type out of 'arithmetic' and geometric'")
        pass

In [4]:
def row_asian_option_price(row):
    return  asian_option_price(
        row['spot_price'],
        row['strike_price'],
        row['risk_free_rate'],
        row['dividend_rate'],
        row['w'],
        row['averaging_type'],
        row['fixing_frequency'],
        row['n_fixings'],
        row['past_fixings'],
        row['kappa'],
        row['theta'],
        row['rho'],
        row['eta'],
        row['v0'],
        row['calculation_date']
    )

In [5]:
def df_asian_option_price(df):
    prices = Parallel(n_jobs=-1)(delayed(row_asian_option_price)(row) for _, row in df.iterrows())
    return prices

In [6]:
calculation_datetime = datetime.today()
"""
365.41	548	 0.04	0.0	put	geometric	90	10	0	0.412367	0.17771	-0.582856	0.785592	0.08079	2024-10-18 13:38:09.345440	900	147.330432	156.641565
"""
r = 0.04
g = 0.0

w = 'put'

past_fixings = 0
s,k = 365.41, 548
# s = 365.41
# k = 370.00
kappa = 0.412367
theta = 0.17771
rho = -0.582856
eta = 0.785592
v0 = 0.08079
n_fixings = 10
fixing_frequency = 90

averaging_type = 'geometric'

t = n_fixings*fixing_frequency


price = asian_option_price(s,k,r,g,w,averaging_type,n_fixings,fixing_frequency,past_fixings,kappa,theta,rho,eta,v0,calculation_datetime)
my_price = ms.asian_option_price(s,k,r,g,w,averaging_type,n_fixings,fixing_frequency,past_fixings,kappa,theta,rho,eta,v0,calculation_datetime)
vanilla = ms.ql_heston_price(s,k,t,r,g,w,kappa,theta,rho,eta,v0,datetime.today())

print(f"asian: {price}  {my_price}\nvanilla: {vanilla}")

asian: 155.541142583563  155.541142583563
vanilla: 147.33043221165957


In [7]:
fixing_frequencies = [
    # 1,7,
    30,
    # 90
]
n_fixings = [
    5,
    # 10
]
K = np.unique(np.linspace(s*0.5,s*1.5,15).astype(int))

W = [
    # 'call',
    'put'
]
types = [
    'arithmetic',
    # 'geometric'
]
past_fixings = [0]
features = pd.DataFrame(
    product(
        [s],
        K,
        [r],
        [g],
        W,
        types,
        fixing_frequencies,
        n_fixings,
        past_fixings,
        [kappa],
        [theta],
        [rho],
        [eta],
        [v0],
        [calculation_datetime]
    ),
    columns = [
        'spot_price','strike_price','risk_free_rate','dividend_rate','w',
        'averaging_type','fixing_frequency','n_fixings','past_fixings',
        'kappa','theta','rho','eta','v0','calculation_date'
    ]
)
features['days_to_maturity'] = features['n_fixings']*features['fixing_frequency']

In [8]:
start= time.time()
features['asian_price'] = df_asian_option_price(features)
end = time.time()
print(features[['spot_price', 'strike_price', 'asian_price',]])
details = features[['risk_free_rate', 'dividend_rate', 'w',
       'averaging_type', 'fixing_frequency', 'n_fixings', 'past_fixings',
       'kappa', 'theta', 'rho', 'eta', 'v0', 'calculation_date',
       'days_to_maturity']].drop_duplicates()
print('\n')
for detail in details.columns:
    print(f"{detail}:")
    print(f"     {details[detail].loc[0]}")
print(end-start)

    spot_price  strike_price  asian_price
0       365.41           182     0.001229
1       365.41           208     0.005289
2       365.41           234     0.032478
3       365.41           261     0.167171
4       365.41           287     0.648965
5       365.41           313     2.092963
6       365.41           339     5.703704
7       365.41           365    13.453997
8       365.41           391    27.773739
9       365.41           417    48.904517
10      365.41           443    73.347624
11      365.41           469    98.694863
12      365.41           495   124.224991
13      365.41           522   150.774702
14      365.41           548   176.348829


risk_free_rate:
     0.04
dividend_rate:
     0.0
w:
     put
averaging_type:
     arithmetic
fixing_frequency:
     30
n_fixings:
     5
past_fixings:
     0
kappa:
     0.412367
theta:
     0.17771
rho:
     -0.582856
eta:
     0.785592
v0:
     0.08079
calculation_date:
     2024-10-18 16:50:20.024372
days_to_maturity:
  

In [10]:
features['asian_price'] = asian_option_pricer.df_asian_option_price(features)
features

,spot_price,strike_price,risk_free_rate,dividend_rate,w,averaging_type,fixing_frequency,n_fixings,past_fixings,kappa,theta,rho,eta,v0,calculation_date,days_to_maturity,asian_price
0,365.41,182,0.04,0.0,put,arithmetic,30,5,0,0.412367,0.17771,-0.582856,0.785592,0.08079,2024-10-18 16:50:20.024372,150,0.000384
1,365.41,208,0.04,0.0,put,arithmetic,30,5,0,0.412367,0.17771,-0.582856,0.785592,0.08079,2024-10-18 16:50:20.024372,150,0.004266
2,365.41,234,0.04,0.0,put,arithmetic,30,5,0,0.412367,0.17771,-0.582856,0.785592,0.08079,2024-10-18 16:50:20.024372,150,0.033138
3,365.41,261,0.04,0.0,put,arithmetic,30,5,0,0.412367,0.17771,-0.582856,0.785592,0.08079,2024-10-18 16:50:20.024372,150,0.177233
4,365.41,287,0.04,0.0,put,arithmetic,30,5,0,0.412367,0.17771,-0.582856,0.785592,0.08079,2024-10-18 16:50:20.024372,150,0.683313
5,365.41,313,0.04,0.0,put,arithmetic,30,5,0,0.412367,0.17771,-0.582856,0.785592,0.08079,2024-10-18 16:50:20.024372,150,2.162196
6,365.41,339,0.04,0.0,put,arithmetic,30,5,0,0.412367,0.17771,-0.582856,0.785592,0.08079,2024-10-18 16:50:20.024372,150,5.794424
7,365.41,365,0.04,0.0,put,arithmetic,30,5,0,0.412367,0.17771,-0.582856,0.785592,0.08079,2024-10-18 16:50:20.024372,150,13.584606
8,365.41,391,0.04,0.0,put,arithmetic,30,5,0,0.412367,0.17771,-0.582856,0.785592,0.08079,2024-10-18 16:50:20.024372,150,27.903593
9,365.41,417,0.04,0.0,put,arithmetic,30,5,0,0.412367,0.17771,-0.582856,0.785592,0.08079,2024-10-18 16:50:20.024372,150,49.040082
